In [1]:
# Time to do a better job of using SCOP

#Turn out scop provides a "SCOP id" that probably makes it easier to use the API

SH2_ID=4001650
A2A_ID = 8091190
#CLASS_A_GPCR_ID=4001858

SCOPID = SH2_ID

In [2]:
import pandas as pd
import time
import os
import glob
from selenium import webdriver
from prody import *
import shutil
import requests, bs4, re, time

import MDAnalysis as mda
import numpy as np
import pyemma as pe
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import Bio.PDB as bpdb
import nglview as nv
from pylab import *
startdir = os.getcwd()

_ColormakerRegistry()

In [3]:
def getSCOP(SCOPid):

    driver = webdriver.Chrome()
    driver.get('http://scop.mrc-lmb.cam.ac.uk/term/'+str(SCOPid))
    like = driver.find_elements_by_class_name('icon')

    for x in range(0,len(like)):
        page_before_click = str(bs4.BeautifulSoup(driver.page_source, 'lxml'))
        like[x].click()
        page_after_click = str(bs4.BeautifulSoup(driver.page_source, 'lxml'))
        while page_before_click == page_after_click:
            like[x].click()
            page_after_click = str(bs4.BeautifulSoup(driver.page_source, 'lxml'))

    jspagedata = bs4.BeautifulSoup(driver.page_source, 'lxml')
    driver.close()
    with open("scop_"+str(SCOPid)+".csv","a")as csvfile:
        csvfile.write("entryname,region,pdbid,chainid\n")
    td_containers=jspagedata.findAll('div',class_ = "td-container")
    for i in range(len(td_containers)):
        container = td_containers[i]
        if len(container.findAll("div",class_="grandchildrenlist")) > 0 :
            entryname = str(container.find("div",class_ = "td-protein-line")).split("/span>")[1].split("</div")[0].replace(",",";")
            region = str(container.findAll("div",class_="td-protein-region")[1]).split(':')[1].split('</')[0]
            pdb_ids_with_chain = (" " +str(container.findAll("div",class_="grandchildrenlist")[0].findAll("div")).replace("[","").replace("]","").replace("<div>","").replace("</div>","")).split(",")
            pdb_ids_in_this_container=[]
            chain_ids_in_this_container=[]
            for ind in range(len(pdb_ids_with_chain)):
                pdb_ids_in_this_container.append(pdb_ids_with_chain[ind].split(" ")[1])
                chain_ids_in_this_container.append(pdb_ids_with_chain[ind].split(" ")[2])
                csvrow = entryname+","+str(region)+","+pdb_ids_with_chain[ind].split(" ")[1]+"," + pdb_ids_with_chain[ind].split(" ")[2]+"\n"
                with open("scop_"+str(SCOPid)+".csv","a")as csvfile:
                    csvfile.write(csvrow)
    return pd.read_csv("scop_"+str(SCOPid)+".csv")

In [4]:
try:
    os.mkdir(startdir+"\\"+str(SCOPID))
    os.chdir(startdir+"\\"+str(SCOPID))

except:os.chdir(startdir+"\\"+str(SCOPID))


#Use my webscraping code to get a list of PDBs from SCOP
shutil.copy2(startdir + "\\chromedriver.exe", startdir+"\\"+str(SCOPID)+"\\chromedriver.exe")
scopdf= getSCOP(SCOPID)

# Make it a unique and sorted list
pdbIDs=sorted(list(set(list(scopdf.pdbid))))
# Download all thsoe PDBs
fetchPDB(pdbIDs,compressed = False)
#scopchaindict = dict(zip(scop_pdbs,scop_chains))
pdbs = sorted(list(set(glob.glob(startdir+"\\"+str(SCOPID)+"\\*.pdb"))))
# NExt we will Parse all the PDBs

#To start, i will parse each PDB and keep the ordering consistent w the dataframe (since each PDBs can have multiple chains and that data is only stroes in the form of the dataframe)

structures=[]
headers=[]
for i in range(len(scopdf.pdbid)):
    structure,header = parsePDB(scopdf.pdbid[i],header=True)
    structures.append(structure)
    headers.append(header)



@> PDB file is found in working directory (3n84.pdb).
@> 6278 atoms and 1 coordinate set(s) were parsed in 0.07s.
@> PDB file is found in working directory (3n84.pdb).
@> 6278 atoms and 1 coordinate set(s) were parsed in 0.07s.
@> PDB file is found in working directory (2h5k.pdb).
@> 1631 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (3kfj.pdb).
@> 957 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (1fyr.pdb).
@> 3538 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (2huw.pdb).
@> 1870 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (1fhs.pdb).
@> 1820 atoms and 18 coordinate set(s) were parsed in 0.10s.
@> PDB file is found in working directory (1cj1.pdb).
@> 10060 atoms and 1 coordinate set(s) were parsed in 0.14s.
@> PDB file is found in working directory (2aob.pdb).
@> 4036 atoms and 1 coordinate se

@> PDB file is found in working directory (3s8o.pdb).
@> 960 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (2huw.pdb).
@> 1870 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> PDB file is found in working directory (5cdw.pdb).
@> 25334 atoms and 1 coordinate set(s) were parsed in 0.27s.
@> PDB file is found in working directory (1cj1.pdb).
@> 10060 atoms and 1 coordinate set(s) were parsed in 0.13s.
@> PDB file is found in working directory (2aob.pdb).
@> 4036 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> PDB file is found in working directory (1bmb.pdb).
@> 1011 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (1cj1.pdb).
@> 10060 atoms and 1 coordinate set(s) were parsed in 0.18s.
@> PDB file is found in working directory (3n7y.pdb).
@> 2942 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (1cj1.pdb).
@> 10060 atoms and 1 coordinate 

@> PDB file is found in working directory (6crf.pdb).
@> 7170 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> PDB file is found in working directory (4nwf.pdb).
@> 8670 atoms and 1 coordinate set(s) were parsed in 0.16s.
@> PDB file is found in working directory (6cms.pdb).
@> 7840 atoms and 1 coordinate set(s) were parsed in 0.10s.
@> PDB file is found in working directory (6bmr.pdb).
@> 8695 atoms and 1 coordinate set(s) were parsed in 0.13s.
@> PDB file is found in working directory (6md9.pdb).
@> 8348 atoms and 1 coordinate set(s) were parsed in 0.10s.
@> PDB file is found in working directory (6bmx.pdb).
@> 8564 atoms and 1 coordinate set(s) were parsed in 0.10s.
@> PDB file is found in working directory (6atd.pdb).
@> 7982 atoms and 1 coordinate set(s) were parsed in 0.09s.
@> PDB file is found in working directory (5i6v.pdb).
@> 8918 atoms and 1 coordinate set(s) were parsed in 0.13s.
@> PDB file is found in working directory (4je4.pdb).
@> 1616 atoms and 1 coordinate set

@> PDB file is found in working directory (6mdb.pdb).
@> 8427 atoms and 1 coordinate set(s) were parsed in 0.11s.
@> PDB file is found in working directory (6crg.pdb).
@> 7793 atoms and 1 coordinate set(s) were parsed in 0.09s.
@> PDB file is found in working directory (4h34.pdb).
@> 4041 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (5ehp.pdb).
@> 8564 atoms and 1 coordinate set(s) were parsed in 0.10s.
@> PDB file is found in working directory (5x94.pdb).
@> 3546 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (6crf.pdb).
@> 7170 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> PDB file is found in working directory (4nwf.pdb).
@> 8670 atoms and 1 coordinate set(s) were parsed in 0.16s.
@> PDB file is found in working directory (6cms.pdb).
@> 7840 atoms and 1 coordinate set(s) were parsed in 0.16s.
@> PDB file is found in working directory (6bmr.pdb).
@> 8695 atoms and 1 coordinate set

@> PDB file is found in working directory (1o4g.pdb).
@> 1022 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (1o44.pdb).
@> 1045 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (1o4l.pdb).
@> 1013 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (1o49.pdb).
@> 1027 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (1o4q.pdb).
@> 1020 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (1a1c.pdb).
@> 2242 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> PDB file is found in working directory (1a09.pdb).
@> 2733 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> PDB file is found in working directory (4f5a.pdb).
@> 935 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (3vs2.pdb).
@> 7189 atoms and 1 coordinate set(

@> PDB file is found in working directory (3ob1.pdb).
@> 2686 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (5hkw.pdb).
@> 7511 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> PDB file is found in working directory (3buw.pdb).
@> 5731 atoms and 1 coordinate set(s) were parsed in 0.06s.
@> PDB file is found in working directory (5j3x.pdb).
@> 18284 atoms and 1 coordinate set(s) were parsed in 0.27s.
@> PDB file is found in working directory (5j3x.pdb).
@> 18284 atoms and 1 coordinate set(s) were parsed in 0.29s.
@> PDB file is found in working directory (1b47.pdb).
@> 8092 atoms and 1 coordinate set(s) were parsed in 0.09s.
@> PDB file is found in working directory (4gpl.pdb).
@> 2588 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> PDB file is found in working directory (2y1n.pdb).
@> 6623 atoms and 1 coordinate set(s) were parsed in 0.07s.
@> PDB file is found in working directory (3ob2.pdb).
@> 2848 atoms and 1 coordinate s

@> PDB file is found in working directory (5eel.pdb).
@> 5572 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> PDB file is found in working directory (1mw4.pdb).
@> 2066 atoms and 10 coordinate set(s) were parsed in 0.07s.
@> PDB file is found in working directory (5eeq.pdb).
@> 2027 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> PDB file is found in working directory (5d0j.pdb).
@> 3241 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> PDB file is found in working directory (5u06.pdb).
@> 3886 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> PDB file is found in working directory (4wwq.pdb).
@> 1988 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (5tyi.pdb).
@> 3651 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> PDB file is found in working directory (4x6s.pdb).
@> 1889 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (5u1q.pdb).
@> 3690 atoms and 1 coordinate se

@> PDB file is found in working directory (6amw.pdb).
@> 3938 atoms and 20 coordinate set(s) were parsed in 0.32s.
@> PDB file is found in working directory (1ab2.pdb).
@> 1684 atoms and 20 coordinate set(s) were parsed in 0.13s.
@> PDB file is found in working directory (3k2m.pdb).
@> 3385 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (5mo4.pdb).
@> 3762 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> PDB file is found in working directory (1opl.pdb).
@> 6655 atoms and 1 coordinate set(s) were parsed in 0.16s.
@> PDB file is found in working directory (5dc4.pdb).
@> 1717 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (3t04.pdb).
@> 1736 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> PDB file is found in working directory (3k2m.pdb).
@> 3385 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> PDB file is found in working directory (5dc9.pdb).
@> 1790 atoms and 1 coordinate s

@> PDB file is found in working directory (2etz.pdb).
@> 1857 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (2eu0.pdb).
@> 1857 atoms and 20 coordinate set(s) were parsed in 0.11s.
@> PDB file is found in working directory (1lum.pdb).
@> 1764 atoms and 20 coordinate set(s) were parsed in 0.13s.
@> PDB file is found in working directory (2k7a.pdb).
@> 2749 atoms and 20 coordinate set(s) were parsed in 0.17s.
@> PDB file is found in working directory (1lun.pdb).
@> 1764 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> PDB file is found in working directory (3s9k.pdb).
@> 960 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> PDB file is found in working directory (1a81.pdb).
@> 12201 atoms and 1 coordinate set(s) were parsed in 0.14s.
@> PDB file is found in working directory (4fl2.pdb).
@> 4677 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> PDB file is found in working directory (1a81.pdb).
@> 12201 atoms and 1 coordinate

@> WARNING Discarding model 15, which contains 1 more atoms than first model does.
@> WARNING Discarding model 16, which contains 1 more atoms than first model does.
@> WARNING Discarding model 17, which contains 1 more atoms than first model does.
@> WARNING Discarding model 18, which contains 1 more atoms than first model does.
@> WARNING Discarding model 19, which contains 1 more atoms than first model does.
@> WARNING Discarding model 20, which contains 1 more atoms than first model does.
@> WARNING Discarding model 21, which contains 1 more atoms than first model does.
@> WARNING Discarding model 22, which contains 1 more atoms than first model does.
@> WARNING Discarding model 23, which contains 1 more atoms than first model does.
@> WARNING Discarding model 24, which contains 1 more atoms than first model does.
@> WARNING Discarding model 25, which contains 1 more atoms than first model does.
@> WARNING Discarding model 26, which contains 1 more atoms than first model does.
@> W

In [5]:
# I guess the first Thing I want to do is to try to determine which of the PDBs are ligand bound and which are not
complexes=[]
non_complexes=[]
for i in range(len(headers)):
    if headers[i]["title"].find("COMPLEX") != -1:
        complexes.append(scopdf.pdbid[i])
    else:
        non_complexes.append(scopdf.pdbid[i])
        
    

In [6]:

# make selections for the regions that contain the SH2 domain (according to scop)
my_selections = []
all_selections = []
for i in range(len(scopdf.pdbid)):
    
    sel = "chain " + str(scopdf.chainid[i]) + " and resnum < " + str(scopdf.region[i]).split("-")[1]+ " and resnum > " + str(scopdf.region[i]).split("-")[0]
    all_selections.append(sel)
    if sel!=None:
        my_selections.append(sel)

#for i in range(len(my_selections)):
#    if my_selections[i]!=None:
#        matchAlign(my_selections[0],my_selections[i])

In [7]:
my_ensemble = buildPDBEnsemble(my_selections[:int(len(my_selections)/2)],my_selections[0])

AttributeError: 'str' object has no attribute 'getTitle'

In [ ]:
try: os.mkdir("selections")
except: pass
for i in range(len(scopdf.pdbid)):
    if type(all_selections[i])==None:
        pass
    else:
        try: writePDB("selections/"+scopdf.pdbid[i]+"_"+scopdf.chainid[i]+".pdb",all_selections[i],renumber = True)
        except:pass
    
    

In [ ]:
u=[]
for i in range(len(scopdf.pdbid)):
    u.append(mda.Universe(scopdf.pdbid[i] +".pdb"))
    
    

In [ ]:
len(all_selections)

In [ ]:
type(all_selections[i])